In [1]:
import exploratory_analysis

import json
import uuid
import re
import pickle
import pandas as pd

import plotly.express as px
import kaleido

sys.path.append('src/')
import preprocessing
import get_ent_info
import exploratory_analysis
import get_reference


%load_ext autoreload
%autoreload 2


## 1. Distribution of single variables

### 1.1 Entries (entity as entry)

metadata (title, author, publication date and revision date)

In [ ]:


with open('database/entry_meta_dict.json', 'r') as f:
    ent_dict = json.load(f)

ent_author_list = []
ent_year_list = []

for ent_url in list(ent_dict.keys()): # for each entry
    # author(s)
    author_list = ent_dict[ent_url]['authorship']
    ent_author_list = ent_author_list + author_list
    # year
    pub_year = ent_dict[ent_url]['pubinfo']['first_published_year']
    rev_year = ent_dict[ent_url]['pubinfo']['substantive_revision_year']
    ent_year_list.append((pub_year, rev_year))

# ========================== author

unique_ent_author_list = list(set(ent_author_list))
print("Number of authors:", len(ent_author_list))
print("Number of unique authors:", len(unique_ent_author_list))

# create dict
ent_author_dict, uni_author_id_dict = {}, {}
for author in unique_ent_author_list:
    uni_author_id_dict[author] = str(uuid.uuid4())
    ent_author_dict[uni_author_id_dict[author]] = {
            "name": author,
            "ref_count": 0,
            "ref_list": []
        }
var = "ent_author" # the order matters
with open(f'database/single_variable_list/uni_{var}_id_dict.json', 'w') as f:
    json.dump(ent_author_dict, f, indent=4)

for ent_url in list(ent_dict.keys()):
    author_list = ent_dict[ent_url]['authorship']
    for author in author_list:
        ent_author_dict[uni_author_id_dict[author]]['ref_count'] += 1
        ent_author_dict[uni_author_id_dict[author]]['ref_list'].append(ent_url)

# sort
var_dict = dict(sorted(ent_author_dict.items(), key=lambda item: item[1]["ref_count"], reverse=True))
# save
with open(f'database/single_variable_list/{var}_dict.json', 'w') as f:
    json.dump(var_dict, f, indent=4)

exploratory_analysis.vis_single_var(var, 70)

# ========================== year

unique_ent_year_list = list(set(ent_year_list))
print("Number of pub-rev year pairs:", len(ent_year_list))
print("Number of unique pub-rev year pairs:", len(unique_ent_year_list))

# create dict
# we do not need id here
ent_year_dict = {}
for var in unique_ent_year_list:
    # don't be tuple
    pub_rev_year = var[0]+", "+var[1]
    # uni_author_id_dict[var] = str(uuid.uuid4())
    ent_year_dict[pub_rev_year] = {
            "name": pub_rev_year,
            "ref_count": 0,
            "ref_list": []
        }
var = "ent_year"
with open(f'database/single_variable_list/uni_{var}_id_dict.json', 'w') as f:
    json.dump(ent_author_dict, f, indent=4)

for ent_url in list(ent_dict.keys()):
    pub_year = ent_dict[ent_url]['pubinfo']['first_published_year']
    rev_year = ent_dict[ent_url]['pubinfo']['substantive_revision_year']
    pair_year = pub_year + ", " + rev_year
    # it is not a list!!!!
    ent_year_dict[pair_year]['ref_count'] += 1
    ent_year_dict[pair_year]['ref_list'].append(ent_url)

# sort based on publication year
var_dict = dict(sorted(ent_year_dict.items(), key=lambda item: int(item[1]["name"][:4]), reverse=True))
# sort
var_dict = dict(sorted(ent_year_dict.items(), key=lambda item: item[1]["ref_count"], reverse=True))
# save
with open(f'database/single_variable_list/{var}_dict.json', 'w') as f:
    json.dump(var_dict, f, indent=4)
exploratory_analysis.vis_single_var(var, 60)



In [ ]:

# only for the first publiaction date


# only for the evision date



#### Pub Date

“A Solution to the Problem of Updating Encyclopedias”, by Eric Hammer and Edward N. Zalta, Computers and the Humanities, 31/1 (1997): 47–60. [Note: The ftp-based file upload system described in this paper was superseded by a browser-based file upload system which uses special password-protected web interfaces for the authors and editors.]
“Why Philosophy Needs a ‘Dynamic’ Encyclopedia”, by John Perry and Edward N. Zalta, URL = <https://plato.stanford.edu/pubs/why.html>, November 1997.

#### 1.2.1 Bibliography (Count)

### 1.2 Related Links

### 1.3 Bibliography (year, author, ref_text, title, parent_title)



- `author`: # consider two names refer to the same person
- `title`
    title_list: 186632
    unique_title_list: 152806
    parent_title_list: 186632
    unique_parent_title_list: 32994
- `ref`
    ref_text_list: 186632
    unique_ref_list: 183069
- 

In [ ]:

# filter the ent_ref_dict
# only include those are not primary sources

# count how many entries cite the...


In [ ]:

with open('database/sep_clean/ent_ref_sep_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)

# print keys
for ent_url in list(ent_ref_dict.keys()): # for each entry
    cat_ref_list = list(ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list: # for each categories in bibliogrpahies
        ref_id_list = list(ent_ref_dict[ent_url][cat]["ref_dict"].values()) # get the info dict of each ref
        print(list(ref_id_list[0]))
        break
    break
# Authorship

# var = "author_list"
# is_list = True
# exploratory_analysis.ref_var_count_with_id(ent_ref_dict, var, is_list) # this will create a dict for vis_single_var to open
# exploratory_analysis.vis_single_var(var, 100)

var = "update_author_list"
is_list = True
exploratory_analysis.ref_var_count_with_id(ent_ref_dict, var, is_list) # this will create a dict for vis_single_var to open
exploratory_analysis.vis_single_var(var, 50)

# var = "title_in_title"
# is_list = False
# exploratory_analysis.ref_var_count_with_id(ent_ref_dict, var, is_list) # this will create a dict for vis_single_var to open
# exploratory_analysis.vis_single_var(var, 100)

# var = "title_year"

# is_list = False
# exploratory_analysis.ref_var_count_with_id(ent_ref_dict, var, is_list) # this will create a dict for vis_single_var to open
# exploratory_analysis.vis_single_var(var, 50)

var = "update_ref"
is_list = False
exploratory_analysis.ref_var_count_with_id(ent_ref_dict, var, is_list) # this will create a dict for vis_single_var to open
exploratory_analysis.vis_single_var(var, 50)

var = "year"
is_list = False
exploratory_analysis.ref_var_count_with_id(ent_ref_dict, var, is_list) # this will create a dict for vis_single_var to open
exploratory_analysis.vis_single_var(var, 60)


# for authors cited by entries on a specific author

# for authors cited by entries on a philosophical issue


In [39]:
# produce data summary table

# oepn database\single_variable_list\update_author_list_dict.json
with open('database/single_variable_list/update_author_list_dict.json', 'r') as f:
    info_dict = json.load(f)
# with open('database/single_variable_list/update_ref_dict.json', 'r') as f:
#     info_dict = json.load(f)
# with open('database/single_variable_list/year_dict.json', 'r') as f:
#     info_dict = json.load(f)
# build a df of author and ref count
df = pd.DataFrame.from_dict(info_dict, orient='index')
df = df.sort_values(by=['ref_count'], ascending=False)
df = df.reset_index()
df.head(50)

# produce data summary table
df.describe()


,ref_count
count,72538.000000
mean,3.144972
std,10.648137
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,863.000000


## Relationship between two variables

## Visualizations of the basic statistics

## Build Groups

### Entry type

In [36]:
# from entry_meta_dict get list of ent_type and type_parent
with open('database/entry_meta_dict.json', 'r') as f:
    ent_meta_dict = json.load(f)

# build a df of ent_url, ent_type and type_parent
ent_type_list = []
for ent_url in list(ent_meta_dict.keys()):
    ent_type_list.append([ent_url, ent_meta_dict[ent_url]['ent_type'], ent_meta_dict[ent_url]['type_parent']])
df = pd.DataFrame(ent_type_list, columns=['ent_url', 'ent_type', 'type_parent'])

# sort by ent_type
df = df.sort_values(by=['ent_type'], ascending=True)
df = df.reset_index(drop=True)
# print df
# df.head(1000)

# save to database/ent_meta/
# df.to_csv('database/ent_meta/ent_type.csv', index=False)

# summarize by ent_type, if type_parent is not empty
# df_new = df[df['type_parent'] != '']
# count ignore NaN
df.groupby('ent_type').count()
print(df.groupby('ent_type').count().to_latex())
# summarize by type_parent if it is not empty
# df[df['type_parent'] != '']['type_parent'].value_counts()

# count by ent_type and type_parent
# df = df.groupby(['ent_type', 'type_parent']).count()
# # sort by ent_url
# df = df.sort_values(by=['ent_url'], ascending=False)
# # reset index
# # df = df.reset_index()
# # print df
# df

# # save to database/ent_meta/
# df.to_csv('database/ent_meta/ent_type_parent_sum.csv', index=True)
# print(df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  ent\_url &  type\_parent \\
ent\_type           &          &              \\
\midrule
thinker            &      447 &          447 \\
thought of thinker &      175 &          175 \\
topic              &     1147 &         1147 \\
\bottomrule
\end{tabular}



# Publication Year

In [35]:

# read database/two_variable_dict/ENT_avg_ref_year.csv
df = pd.read_csv('database/two_variable_dict/ENT_avg_ref_year.csv')


# read database/two_variable_dict/CITED_AUTHOR_avg_ref_year.csv 
df = pd.read_csv('database/two_variable_dict/CITED_AUTHOR_avg_ref_year.csv')

# latest rev year
# df = pd.read_csv('database/two_variable_dict/ENT_rev_year.csv')


# sort by avg_ref_year
df = df.sort_values(by=['avg_ref_year'], ascending=False)
# df = df.sort_values(by=['latest_rev_year'], ascending=False)
df = df.reset_index(drop=True)
# print df

# summarize the avg_ref_year
print(df['avg_ref_year'].describe())
# print as latex
# print(df['avg_ref_year'].describe().to_latex())

# visualize the avg_ref_year with px
fig = px.histogram(df, x="avg_ref_year", nbins=2000)
# design
fig.update_layout(
    # title=f"Histogram of entries' average reference publication year",
    title=f"Histogram of average publication year of anthors' cited works",
    # title=f"Histogram of latest revision year of SEP entries",
    xaxis_title="Latest Revision Year",
    yaxis_title="count",
    font=dict(
        family="Merriweather",
        size=12,
        color= "black",
    ),
)
# center the title
fig.update_layout(title_x=0.5)
# set tickmode='linear'
# fig.update_xaxes(tickmode='linear')

# fig.update_traces(marker_color="rgba(70,130,180, 0.5)")
fig.update_traces(marker_color="rgba(160,139,194,0.6)")
# reset xaxis range
fig.update_xaxes(range=[1400, 2030])

# beautify the design, old school style
fig.update_layout(
    margin=dict(l=20, r=20, t=50, b=20),
    plot_bgcolor="rgba(0,0,0,0)",
    paper_bgcolor="rgba(220,220,220, 0.2)",
    # paper_bgcolor="rgba(250,235,210, 0.3)"
)
fig.show()
# save as png
# fig.write_image("database/two_variable_dict/ENT_avg_ref_year.png")
fig.write_image("database/two_variable_dict/CITED_AUTHOR_avg_ref_year.png")
# fig.write_image("database/two_variable_dict/CITED_AUTHOR_avg_ref_year.png")


# bar chart





count    71616.000000
mean      1991.206883
std         50.784684
min       1000.000000
25%       1989.000000
50%       2003.000000
75%       2011.000000
max       2022.000000
Name: avg_ref_year, dtype: float64


### Primary and secondary sources